In [4]:
import os
import csv
import sys
import openpyxl
import time
os.chdir(r'C:\Users\op1\Desktop\Pyfile\上手風控\data')
class productInfo:
    equitypsi = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    marginpsi = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    def __init__(self, contract=None, month=None,exchange=None,
                 Buy_oi=0,Sell_oi=0,tick=[0,0],margin=0,setP=0,currency=None,
                TDate=None,Broker='PHI'):
        
        self.contract = contract
        self.month = month
        self.exchange = exchange
        self.Buy_oi = Buy_oi
        self.Sell_oi = Sell_oi
        self.tick = tick
        self.margin = margin
        self.setP = setP
        self.currency = currency
        self.TDate = TDate
        self.Broker = Broker

    def change(self):
        with open('商品轉換.csv','rt',encoding = 'utf-8') as fphi:
            phifile = csv.reader(fphi)
            check_product = 0
            for i in phifile:
                if self.contract == i[0]:
                    self.contract = i[1]
                    self.exchange = i[2]
                    self.currency = i[3]
                    self.tick[0] = i[4]
                    self.tick[1]= i[5]
                    check_product += 1  
            if check_product == 0 :
                print('請確認',self.contract,'找不到對照商品')
    def pimargin(self):
        with open('//10.36.205.170/交易結算部/國外/交易量+未平倉/' +date[5:]+'/保證金.TXT','r') as mar:
            marR = mar.readlines()
            for i in marR:
                if len(i.split())==11 and i.split()[2]==self.contract:
                    self.margin = int(i.split()[6].replace('.00','').replace(',',''))
                    
    def proportion(self,currency):
        usdS = []
        usdS1 = [] # 占比小於1%
        usdN =[]
        for i in psiOI:
            pi = productInfo(i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7])
            marALL = int(float(i[3].replace(',','').replace('-',''))*float(i[5].replace(',','')))
            marPPT = round(marALL/float(productInfo.equitypsi[i[7]].replace(',','')),4)
            if i[7]==currency:
                if marPPT>.01:
                    usdN.append(i[0]+'-'+i[1])
                    usdS.append(marALL)
                else:
                      usdS1.append(marALL)
        usdother = sum(usdS1)
        usdleft =float(productInfo.equitypsi[currency].replace(',',''))-float(productInfo.marginpsi[currency].replace(',',''))
        usdS.insert(0,usdother)
        usdS.insert(0,usdleft)      
        usdN.insert(0,'other')
        usdN.insert(0,'marginLeft')
        return[usdS,usdN]


        
    def pie(data1,data2,currency):     
        plt.figure(figsize=(6,3))    # 顯示圖框架大小
        plt.pie(data1,                           # 數值
                labels = data2,                # 標籤
                autopct = "%1.2f%%",            # 將數值百分比並留到小數點一位
                startangle=90,
#                 textprops = {"fontsize" : 12},  # 文字大小
                shadow=False)                    # 設定陰影
        plt.axis('equal')                                          # 使圓餅圖比例相等
        plt.title('PHI '+currency+" proportion", {"fontsize" : 18})  # 設定標題及其文字大小
#         plt.legend(loc = "best")                                   # 設定圖例及其位置為最佳
        plt.savefig(currency+" PHI proportion.jpg",   # 儲存圖檔
                    bbox_inches='tight',               # 去除座標軸占用的空間
                    pad_inches=0)                    # 去除所有白邊

        
    
    def risk(contract,month,ticks=0):
        for i in psiOI:
            if contract in i and month in i:
                profit = int(i[3])*int(i[4][0])*ticks/float(i[4][1])
        return(profit)
    
    def save(date):
        f1 = open(date.replace('/','-')+' psiOI.csv','w+')
        for i in psiOI:
            i.remove(i[4])
            f1.write(','.join(i)+'\n')
        f1.close()

    def piINFO(self):
        return([
            self.Broker ,
            self.contract,
            self.month,
            self.exchange, 
            self.Buy_oi,
            self.Sell_oi,
            #self.tick , 
            self.margin,
            abs((self.Buy_oi - self.Sell_oi) * self.margin),
            self.setP,
            self.currency
        ])


#--------------------------------------------------上手資料   

fileList = os.listdir()
fileList = [i for i in os.listdir() if i[-3:] == 'txt' and 'PSI' in i]
for i in fileList:
    psi= open(i, "r")
    psiR = psi.readlines()
    psi.close()
    ct = 0
    date = ''
    sourcepsi,sourcepsi = [],[]
    psiOI=[]
    productInfo.equityifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    productInfo.marginifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}

    exchange = ['CBOE','CBOT','CME','COMEX','NYBOT','NYMEX','SGX']
    # 找到 OPEN POSITION 
    for l in range(len(psiR)):
        if 'OPEN' in psiR[l]:
            ct = l
            break
    for i in range(ct,len(psiR)):
        sourcepsi.append(psiR[i].split())

    for i in filter(None, sourcepsi):
        if 'Date' in i:
            date =i[-1]
            date0 = time.strptime(date, '%d/%m/%Y')
            date1 = time.strftime("%Y/%m/%d",date0)   #資料日期
            date = time.strftime("%Y-%m%d",date0)     #sheets名稱資料日期
    for i in filter(None, sourcepsi):
        if len(i)>1:
            pi = productInfo()
            if i[0]=='Initial':
                productInfo.marginpsi['USD'] = int(i[4][:-3].replace(',',''))
                try:
                    productInfo.marginpsi['SGD'] = int(i[3][:-3].replace(',',''))
                except ValueError:
                    productInfo.marginpsi['SGD'] = 0
                try:
                    productInfo.marginpsi['JPY'] = int(i[5][:-3].replace(',',''))
                except ValueError:    
                    productInfo.marginpsi['JPY'] = 0

            if i[1]=='Balance':
                productInfo.equitypsi['USD'] = int(i[4][:-3].replace(',',''))
                try:
                    productInfo.equitypsi['SGD'] = int(i[3][:-3].replace(',',''))
                except ValueError:
                    productInfo.equitypsi['SGD'] = 0
                try:
                    productInfo.equitypsi['JPY'] = int(i[5][:-3].replace(',',''))
                except ValueError:    
                    productInfo.equitypsi['JPY'] = 0


            if len(i)==6:
                if i[1] in exchange:
                    pi.Buy_oi = 0
                    pi.Sell_oi = 0
                    pi.contract = i[2]
                    pi.month = i[3]+i[4]
                    if '-'in i[0]:
                        pi.Sell_oi = int(i[0].replace(',',''))
                    else:
                        pi.Buy_oi = int(i[0].replace(',',''))
                    pi.setP = i[5]
                    pi.change()
                    pi.pimargin()
                    psiOI.append(pi.piINFO())
            elif len(i)==7:
                if i [1] in exchange:
                    pi.contract = i[2]
                    pi.month = i[4]+i[5]
                    if '-'in i[0]:
                        pi.Sell_oi = int(i[0].replace(',',''))
                    else:
                        pi.Buy_oi = int(i[0].replace(',',''))
                    pi.setP = i[6]
                    pi.change()
                    pi.pimargin()
                    psiOI.append(pi.piINFO())
    curL = ['USD','JPY','EUR']
    for cur in curL:
        usdS = []
        usdN = []
        for i in psiOI:
            if i[-1] ==cur:
                usdS.append(i[-3])
                usdN.append(i[1]+i[2])

        for i in range(len(psiOI)):
            if cur in psiOI[i]:
                marPPS = psiOI[i][-3]/sum(usdS)
                psiOI[i].append(marPPS)
                psiOI[i].append(int(psiOI[i][-1]*productInfo.marginpsi[cur]))
    



        
              

    wb = openpyxl.load_workbook('RiskControl.xlsx')

    title=['上手','商品','月份','交易所','買方口數',
           '賣方口數','保證金','加總','結算價','幣別','占比','調整後保證金']

    phiEqtAndMar = [
                    date1, pi.Broker,
                    productInfo.equitypsi['USD'],productInfo.equitypsi['EUR'],
                    productInfo.equitypsi['JPY'],productInfo.equitypsi['HKD'],
                    productInfo.equitypsi['SGD'],
                    productInfo.marginpsi['USD'],productInfo.marginpsi['EUR'],
                    productInfo.marginpsi['JPY'],productInfo.marginpsi['HKD'],
                    productInfo.marginpsi['SGD'],                
                   ]


    try:                      #判斷sheet是否存在
        ws = wb[date]
    except KeyError:
        wb.create_sheet(date)
        ws = wb[date]
        ws.append(title)

    t = 0                     #判斷資料是否存在
    for i in ws['A']:
        if 'PHI' in i.value:
            t=1
            print(date+ '已有資料')
            break       
    if t == 0 :               #資料填入
        for row in range(len(psiOI)):
            ws.append(psiOI[row])

    ##....填回歷史資料

    ws = wb["HistoryData"]

    t = 0
    for i in ws['A']:         #判斷資料是否存在
        if date1 in i.value:
            if 'PHI' in ws.cell(row = i.row , column = 2).value:
                t=1
                break        
    if t == 0 :               #資料填入
        ws.append(phiEqtAndMar)        

    wb.save('RiskControl.xlsx')
    print(pi.Broker+date+' OK')


請確認 FV 找不到對照商品
請確認 TY 找不到對照商品
PHI2021-0301 OK
請確認 FV 找不到對照商品
請確認 TY 找不到對照商品
PHI2021-0302 OK
請確認 TY 找不到對照商品
PHI2021-0303 OK
請確認 TY 找不到對照商品
PHI2021-0305 OK
請確認 TY 找不到對照商品
PHI2021-0308 OK
請確認 TY 找不到對照商品
PHI2021-0309 OK
請確認 TY 找不到對照商品
PHI2021-0310 OK
請確認 TY 找不到對照商品
PHI2021-0311 OK
請確認 TY 找不到對照商品
請確認 PA 找不到對照商品
PHI2021-0312 OK
請確認 TY 找不到對照商品
PHI2021-0315 OK
請確認 TY 找不到對照商品
PHI2021-0316 OK
請確認 TY 找不到對照商品
PHI2021-0317 OK
PHI2021-0319 OK


FileNotFoundError: [Errno 2] No such file or directory: '//10.36.205.170/交易結算部/國外/交易量+未平倉/0322/保證金.TXT'